# PTDF & LODF Sensitivity Analysis

Power Transfer Distribution Factors (PTDFs) and Line Outage Distribution
Factors (LODFs) quantify how branch flows respond to injections and
outages. This notebook demonstrates computing PTDFs for a seller-buyer
transfer, LODFs for a critical branch outage, identifying the most
sensitive branches, and using injection vectors for multi-bus transfers.

Import the case and instantiate the `PowerWorld`.

```python
import numpy as np
from esapp import PowerWorld
from esapp.components import *
from examples.injection import InjectionVector

pw = PowerWorld(case_path)
```

In [ ]:
import numpy as np
from esapp import PowerWorld
from esapp.components import *
from examples.injection import InjectionVector
import ast

with open('../data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

pw = PowerWorld(case_path)

In [ ]:
import sys; sys.path.insert(0, '..')
from plot_helpers import plot_ptdf, plot_lodf, plot_branch_loading

## 1. Base Case & System Overview

Solve the base case and inspect the system before computing sensitivities.

In [ ]:
pw.pflow()
s = pw.summary()
print(f"Buses: {s['n_bus']}  Branches: {s['n_branch']}  Generators: {s['n_gen']}")
print(f"Total gen: {s['total_gen_mw']:.1f} MW   Total load: {s['total_load_mw']:.1f} MW")
print(f"Voltage range: {s['v_min']:.4f} - {s['v_max']:.4f} pu")

## 2. Power Transfer Distribution Factors

PTDFs describe the fraction of a seller→buyer power transfer that flows
through each branch. A PTDF of 0.25 means 25% of the transferred power
flows through that line.

In [ ]:
# Pick two buses as seller and buyer
buses = pw[Bus, 'BusNum']['BusNum']
seller_bus = int(buses.iloc[0])
buyer_bus = int(buses.iloc[-1])
print(f"PTDF: Bus {seller_bus} (seller) → Bus {buyer_bus} (buyer)")

ptdf_df = pw.ptdf(seller_bus, buyer_bus, method='DC')
print(f"\nBranches with |PTDF| > 0.10: {(ptdf_df['LinePTDF'].abs() > 0.10).sum()}")
print(f"Max |PTDF|: {ptdf_df['LinePTDF'].abs().max():.4f}")
ptdf_df.head()

In [ ]:
plot_ptdf(ptdf_df)

## 3. Line Outage Distribution Factors

LODFs describe how flow redistributes across the network when a branch
trips. An LODF of 0.30 means that 30% of the outaged branch's pre-outage
flow redirects through that line.

In [ ]:
# Pick the most loaded branch for the outage study
flows = pw.flows()
most_loaded_idx = flows['LinePercent'].idxmax()
branch_key = (
    int(flows.loc[most_loaded_idx, 'BusNum']),
    int(flows.loc[most_loaded_idx, 'BusNum:1']),
    str(flows.loc[most_loaded_idx, 'LineCircuit']),
)
print(f"LODF study: outage of branch {branch_key[0]}-{branch_key[1]} ckt {branch_key[2]}")
print(f"Pre-outage flow: {flows.loc[most_loaded_idx, 'LineMW']:.1f} MW "
      f"({flows.loc[most_loaded_idx, 'LinePercent']:.1f}% loaded)")

lodf_df = pw.lodf(branch_key, method='DC')
print(f"\nBranches with |LODF| > 0.05: {(lodf_df['LineLODF'].abs() > 0.05).sum()}")
lodf_df.head()

In [ ]:
plot_lodf(lodf_df)

## 4. Post-Outage Flow Estimate

Using the LODF, we can estimate how branch flows change after the outage
without re-solving power flow:

$$\Delta f_k = \text{LODF}_k \times f_{\text{outaged}}$$

In [ ]:
outaged_flow = flows.loc[most_loaded_idx, 'LineMW']
estimated_delta = lodf_df['LineLODF'] * outaged_flow
estimated_post = flows['LineMW'] + estimated_delta

# Show branches with largest estimated flow increase
biggest_increase = estimated_delta.abs().nlargest(5)
print("Top 5 branches by estimated flow change:")
for idx in biggest_increase.index:
    print(f"  {int(flows.loc[idx, 'BusNum'])}-{int(flows.loc[idx, 'BusNum:1'])}: "
          f"{flows.loc[idx, 'LineMW']:.1f} → {estimated_post.loc[idx]:.1f} MW "
          f"(LODF={lodf_df.loc[idx, 'LineLODF']:.3f})")

## 5. Multi-Bus Injection Vectors

The `InjectionVector` class constructs normalized injection patterns
for multi-bus transfers, automatically accounting for system losses.

In [ ]:
bus_df = pw[Bus, ['BusNum']]
inj = InjectionVector(bus_df, losscomp=0.05)

# First two buses supply, last two demand
supply_buses = [int(buses.iloc[0]), int(buses.iloc[1])]
demand_buses = [int(buses.iloc[-1]), int(buses.iloc[-2])]

inj.supply(*supply_buses)
inj.demand(*demand_buses)

alpha = inj.vec
print(f"Injection vector: {len(alpha)} buses")
print(f"  Supply buses {supply_buses}: alpha = {alpha[alpha > 0]}")
print(f"  Demand buses {demand_buses}: alpha = {alpha[alpha < 0]}")
print(f"  Sum (should be ~0 with loss comp): {alpha.sum():.6f}")

## 6. Branch Loading Under Stress

Combine PTDF sensitivity with `snapshot()` to quickly compare branch
loading before and after a transfer increase.

In [ ]:
# Identify most sensitive branch from PTDF
critical_branch = ptdf_df['LinePTDF'].abs().idxmax()
print(f"Most PTDF-sensitive branch: "
      f"{int(ptdf_df.loc[critical_branch, 'BusNum'])}-"
      f"{int(ptdf_df.loc[critical_branch, 'BusNum:1'])}")
print(f"  PTDF = {ptdf_df.loc[critical_branch, 'LinePTDF']:.4f}")

# Current loading
base_loading = pw.flows()
print(f"  Current loading: {base_loading.loc[critical_branch, 'LinePercent']:.1f}%")

overloaded = pw.overloads(threshold=90.0)
print(f"\nBranches above 90% loading: {len(overloaded)}")
if len(overloaded) > 0:
    print(overloaded[['BusNum', 'BusNum:1', 'LineMW', 'LinePercent']].head())